Clase 11
===

Objetivos

* Implementación de dFBA en python.

 En FBA dinámico (en adelante dFBA por su acrónimo en inglés: dynamic FBA) la evolución de la concetración de los metabólitos intracelulares durante la fermentación modifica las restricciones del FBA, cuyo resultado a su vez modifica los flujos de consumo y producción de las ecuaciones cinéticas como se muestra en la figura de [Sanches et al](http://www.sciencedirect.com/science/article/pii/S1096717614000950).

<img src="https://ars.els-cdn.com/content/image/1-s2.0-S1096717614000950-gr2.jpg" alt="Drawing" style="width: 500px;"/>

Usando el trabajo de [Sanches et al](http://www.sciencedirect.com/science/article/pii/S1096717614000950) en *Saccharomyces cerevisiae* como ejemplo, tenemos la siguiente implementación en Python:

In [1]:
# Data based on:
# https://doi.org/10.1016/j.ymben.2014.07.004
# Libraries for the Dynamic block
import numpy as np
from scipy.integrate import odeint
# Libraries for the Metabolic block
import cobra
import os
from os.path import join
data_dir="."
model=cobra.io.read_sbml_model(join(data_dir, "iMM904.xml"))
# Plotting libraries
import matplotlib.pyplot as plt

In [2]:
# Kinetic block
def kineticBlock(Glu,Eth):
    # Parameters
    vGmax, K_Glu, K_Eth = 22.5, 0.88, 6.74
    f_Eth,f_Gly, f_Cit, f_Lac = .112, .273, .169, .137
    v_Glu=vGmax*Glu/(K_Glu+Glu)*1/(1+Eth/K_Eth)
    LB_Eth = v_Glu*f_Eth
    LB_Gly = v_Glu*f_Gly
    LB_Cit = v_Glu*f_Cit
    LB_Lac = v_Glu*f_Lac
    #LB_ATP_Ex = m_ATP
    return [v_Glu,LB_Eth,LB_Gly,LB_Cit,LB_Lac]

# Metabolic block
def metabolicBlock(v_Glu,LB_Eth,LB_Gly,LB_Cit,LB_Lac):
    model.reactions.get_by_id("EX_glc__D_e").lower_bound = -v_Glu # set flux of glucose
    model.reactions.get_by_id("EX_glc__D_e").upper_bound = -v_Glu # set flux of glucose
    # The same for Eth, Gly, Cit, Lac
    solution = model.optimize()
    u = solution.f
    v_Eth = model.reactions.get_by_id("EX_etoh_e").x
    v_Gly = model.reactions.get_by_id("EX_glyc_e").x
    v_Cit = model.reactions.get_by_id("EX_cit_e").x
    v_Lac = model.reactions.get_by_id("EX_lac__D_e").x
    return [u, v_Eth, v_Gly, v_Cit, v_Lac]
    
# Dynamic block
def f(y,t,params):
    V,VX,VGlu,VEth,VGly,VCit,VLac = y # Current values
    F,u, v_Glu, v_Eth, v_Gly, v_Cit, v_Lac = params  # unpack parameters
    Glu_F = 100 # Glucose concentration on feed
    MW_Glu,MW_Eth,MW_Gly,MW_Cit,MW_Lac = [0.18,.046,.092,.192,.090] #Molecular weights
    derivs=[F,                  # dV/dt
            u*VX,                # dVX/dt
            F*Glu_F-v_Glu*MW_Glu*(VX), # dVGlu/dt
            v_Eth*MW_Eth*(VX), # dVEth/dt
            v_Gly*MW_Gly*(VX), # dVGly/dt
            v_Cit*MW_Cit*(VX), # dVCit/dt
            v_Lac*MW_Lac*(VX)] # dVLac/dt
    return derivs
def dynamicBlock(y,params, ti,tf):
    time=np.linspace(ti,tf,100)
    #F,u,v_Glu, v_Eth, v_Gly, v_Cit, v_Lac = params
    soln = odeint(f,y,time,args=(params,))
    # Get solutions at the final time point (tf):
    V=soln[-1,0]
    X,Glu,Eth,Gly,Cit,Lac=soln[-1,1:7]/V
    return [V,X,Glu,Eth,Gly,Cit,Lac]    


# Miscelaneous functions
# Feed flow 
def F(t):
    V_0 =   0.5
    X_0 =   0.5
    Ysx =   0.469
    ti = 0
    SF  = 300
    A,B,C=[0.07,0.07,0.14]
    u_set=A+B*np.exp(-C*t)
    integral = A*(t-ti) + B/C*(np.exp(-C*ti)-np.exp(-C*t)) 
    F= u_set/(SF*Ysx) * V_0*X_0 * np.exp(integral)
    return F
# Save results along the fermentation path
u_path,V_path,X_path = [],[],[]
Glu_path,Eth_path,Gly_path,Cit_path,Lac_path = [],[],[],[],[]
v_Glu_path=[]
def savePath(u,V,X,Glu,Eth,Gly,Cit,Lac,v_Glu):
    global u_path,V_path,X_path
    global Glu_path,Eth_path,Gly_path,Cit_path,Lac_path
    global v_Glu_path
    u_path += [u]
    V_path += [V]
    X_path += [X]
    Glu_path += [Glu]
    Eth_path += [Eth]
    Gly_path += [Gly]
    Cit_path += [Cit]
    Lac_path += [Lac]
    v_Glu_path += [v_Glu]

In [3]:
# Initial conditions
Glu=5
Eth,Gly,Cit,Lac=[0,0,0,0]
V,X=[0.5,0.5]

# Running the simulation over time
time=np.linspace(0,22,200)
for i in range(len(time)):
    # KINETIC BLOCK: 
    # given the current concentrations of glucose (Glu) and ethanol (Eth)
    # we compute the fluxes of Glu, and upper and lower limits of Eth, Glycerol(Gly), Citrate(Cit), and Lactate(Lac)
    v_Glu,LB_Eth,LB_Gly,LB_Cit,LB_Lac = kineticBlock(Glu,Eth)
    # METABOLIC BLOCK
    # given v_Glu,LB_Eth,LB_Gly,LB_Cit,LB_Lac
    # we compute biomass growth rate (u) and metabolic fluxes of extracellular metabolites
    u,v_Eth, v_Gly, v_Cit, v_Lac = metabolicBlock(v_Glu,LB_Eth,LB_Gly,LB_Cit,LB_Lac)
    # DYNAMIC BLOCK
    # given u, V, fluxes (v_Eth, v_Gly, v_Cit, v_Lac), and concentrations (X,Glu,Eth,Gly,Cit,Lac)
    # we update reaction volume V, X and Glu,Eth,Gly,Cit,Lac
    if i==len(time)-1: continue
    y = [V,X*V,Glu*V,Eth*V,Gly*V,Cit*V,Lac*V]
    params = [F(time[i]),u,v_Glu, v_Eth, v_Gly, v_Cit, v_Lac]
    V,X,Glu,Eth,Gly,Cit,Lac = dynamicBlock(y, params, time[i],time[i+1])
    # Save results along the fermentaion path
    savePath(u,V,X,Glu,Eth,Gly,Cit,Lac,v_Glu)

Ejercicios
------------

1. Grafica los perfiles de concentración de biomasa glucosa, y etanol a lo largo del tiempo de fermentación.
2. Repite los resultados anteriores pero esta vez bajo las siguientes condiciones: Glu_0=5,Glu_F=300 y Glu_0=5,Glu_F=100
3. Transforma tu código en un scrip y subelo a tu cuenta de github.